# How to connect to Facebook's Marketing API to manage Marketing campaigns

The home of Facebook's developer environment is its Marketing API. It allows to query data, create and manage ads, and perform a wide variety of other tasks.

Check Facebook's documentation pages here and follow through the steps to get you started: https://developers.facebook.com/docs/marketing-apis/overview


"facebookads" is the Facebook's Python package that provides an interface with Facebook's Marketing API. Find the docs here:
https://developers.facebook.com/docs/marketing-api/reference/sdks/python/v9.0

## get started

https://developers.facebook.com/docs/marketing-apis/get-started

In [1]:
# install
#!pip install facebook_business

In [2]:
import os
os.chdir('/Users/siegstedt/projects/kreativkopf')

### connection details

In [13]:
# App ID und Schlussel
app_id = "443369083543507"
app_secret = "34fe5c99808654e404bf61b172e55fa9"

# id der verknupften seite
page_id = 1610651629161752

# fb add account id des Sandbox accounts
sandbox_id = "act_149874747013728"

# Zugriffsschlussel zum Sandbox Werbeaccount
sandbox_token = 'EAAGTPei6n9MBANPtSCpKmdoaYtjNzmAxk494zzYubuGf7qJrpS5aHWhLQCjZACuoM6JqHofsuUAhcJifn8rrSRE3zHvvh2WBabmuu3OZAzLOpZBZAQOZBrzNPRlNP4IQ11zQdWN0ZCCe5iqbi6U2QhzQy1pLooIKBVORHADPVAJLGZBGTHZCgIjJAbNwAihnb00ZD'

### generate a long lived access token

In [3]:
import src.facebook_auth as fb_auth

In [4]:
# get latest access token
access_token = fb_auth.get_latest_token()

# report
print('Langlebiger Token verfügbar mit der Kennung',access_token[:10],'...',access_token[-10:])

Langlebiger Token verfügbar mit der Kennung EAAGTPei6n ... ctLSEgZDZD


In [9]:
# provide identification data and short lived access token
id_dict = {
    'app_id': "443369083543507",
    'app_secret': "34fe5c99808654e404bf61b172e55fa9",
    'short_lived_token': "EAAGTPei6n9MBAO8MrXboaNjh0bSogCP8atnuO46ycDpWy5dqKwjHAJp28GOLZAb2I0BYbhLTsIDooSUPZBqcZAdsbsnAYa0I12HCDWHvNkwbIkKhK1IGIBMHqu4NIsHZCnWN3e9W3qNAB7eVPOa1Rruvr7romE60HvP1ZCEOWWKlijQA9lLkVG3wsGx1AxJUZD",
}

# generate a long lived token
long_lived_token_dict = {
    'access_token': 'EAAGTPei6n9MBAPxZCDyaOC4vXqEoCinE8gNoyZC3W6nDYmv4WNUlgwG00jExVgZAZCBiagoCVssg2A4Rrjo3RFNMwVZAwYef9Qo0D8KDfVSHXAO2vMLtQVHyOnxREyxvD1UsBJdZAEthdZCAqZAO1ZA2VyCsgzP56VqF0bQd7ctLSEgZDZD',
    'token_type': 'bearer',
    'expires_in': 5184000,
}
#long_lived_token_dict = fb_auth.retrieve_long_lived_token(id_dict['app_id'], id_dict['app_secret'], id_dict['short_lived_token'])

# store token data
fb_auth.store_token(id_dict, long_lived_token_dict)

Token data attached to auth/facebooktoken.txt


### find your target location

Check the following docs for a complete introduction into geographical search options.

https://developers.facebook.com/docs/marketing-api/audiences/reference/targeting-search

The following example will focus on retrieving a location key for a given city. In my case, i am searching for 'Heimbuchenthal'.

In [11]:
import src.facebook_targeting as fb_targeting

search_query = 'Heimbuchenthal'
location_dict = fb_targeting.load_target_location(access_token, search_query)

In [12]:
location_name = location_dict['data'][0]['name']
location_key = location_dict['data'][0]['key']

print(f"Search for {search_query} gave us {len(location_dict['data'])} result(s).")
print(f"Best match was {location_name} with key value {location_key}.")

Search for Heimbuchenthal gave us 1 result(s).
Best match was Heimbuchenthal with key value 561846.


### load FB interest groups

In [5]:
import src.facebook_targeting as fb_targeting
search_query = 'Mode'
interest_dict = fb_targeting.load_target_interest(access_token, search_query)

In [6]:
import pandas as pd
df = pd.DataFrame(interest_dict['data']).sort_values(by='audience_size', ascending=False)
df

,id,name,audience_size,path,description,disambiguation_category,topic
1,6003167425934,Einkaufen und Mode,1871401270,"[Interessen, Einkaufen und Mode]",,NaN,Shopping and fashion
2,6003348604581,Modeaccessories,1150356630,"[Interessen, Einkaufen und Mode, Modeaccessories]",,NaN,Shopping and fashion
3,6003266266843,Modedesign,408515142,"[Interessen, Branchen, Design, Modedesign]",,NaN,Shopping and fashion
4,6003242966413,Modeblog,109675351,"[Interessen, Zusätzliche Interessen, Modeblog]",None,NaN,Business and industry
5,6003654257078,Model,89908700,"[Interessen, Zusätzliche Interessen, Model]",None,NaN,NaN
6,6003434735570,Tesla Model S,85087940,"[Interessen, Zusätzliche Interessen, Tesla Mod...",None,NaN,Hobbies and activities
7,6002969456892,Moderne,77075340,"[Interessen, Zusätzliche Interessen, Moderne]",None,NaN,Business and industry
0,6003247947499,mode,6466420,"[Interessen, Zusätzliche Interessen, mode]",None,Kunst,NaN


In [7]:
# export as list of dicts
target_interest = [i for i in df[['id','name']].head(3).transpose().to_dict().values()]
target_interest

[{'id': '6003167425934', 'name': 'Einkaufen und Mode'},
 {'id': '6003348604581', 'name': 'Modeaccessories'},
 {'id': '6003266266843', 'name': 'Modedesign'}]

### get an esitmate on your potential reach

In [11]:
def strip_reach_estimate(reach_cursor_obj):
    """
    Strips the AdAccountReachEstimate Cursor object to reach value
    
    Args:
        reach_cursor_obj (Cursor object): AdAccountReachEstimate Cursor object
    
    Returns:
        reach estimate (int): estimated amount of reached users
    """
    string = str(reach_cursor_obj).replace('\n','').replace(r'<AdAccountReachEstimate>','').replace(' ','')[2:-2]
    return int(string[string.find("users")+7:])

In [14]:
def load_reach_estimate(id, targeting_spec):
    """
    
    """
    
    from facebook_business.adobjects.adaccount import AdAccount
    from facebook_business.api import FacebookAdsApi
    
    params = {
        'optimize_goal': 'REACH',
        'targeting_spec': targeting_spec,
    }
    
    FacebookAdsApi.init(access_token=sandbox_token)
    account = AdAccount(id)
    reach_cursor = account.get_reach_estimate(params=params)
    return strip_reach_estimate(reach_cursor)
    

location_key = location_key
target_interest = target_interest
location_radius = 20
age_min = 20
age_max = 30

targeting_spec = {
    'geo_locations': {'cities':[{'key':location_key,'radius':location_radius,'distance_unit':'kilometer'}]},
    'interests': target_interest,
    'age_min': age_min,
    'age_max': age_max,
}

reached_users = load_reach_estimate(sandbox_id,targeting_spec)
reached_users

51000

In [15]:
# report
print('SETUP')
print('> ad account:', sandbox_id)
print('> location search query:', 'Heimbuchenthal')
print('> interest search query:', 'Mode')
print('\n')

print('TARGETING')
print('> location name:', location_name)
print('> location radius:', location_radius, 'km')
print('> target interests:', ', '.join([i['name'] for i in target_interest]))
print('> min age:', age_min)
print('> max age:', age_max)
print('\n')

print('ESTIMATE')
print('> est. amount of reached users:', reached_users)

SETUP
> ad account: act_149874747013728
> location search query: Heimbuchenthal
> interest search query: Mode


TARGETING
> location name: Heimbuchenthal
> location radius: 20 km
> target interests: Einkaufen und Mode, Modeaccessories, Modedesign
> min age: 20
> max age: 30


ESTIMATE
> est. amount of reached users: 51000


### set up campaigns and adds

In [ ]:
from facebook_business.adobjects.adaccount import AdAccount
from facebook_business.adobjects.campaign import Campaign
from facebook_business.adobjects.adcreative import AdCreative
from facebook_business.api import FacebookAdsApi

In [ ]:
# init api connection
FacebookAdsApi.init(access_token=sandbox_token)

# generate a new campaign
fields = []
params = {
  'name': 'Look at this beauty',
  'objective': 'LINK_CLICKS',
  'status': 'PAUSED',
  'special_ad_categories': [],
}
print(AdAccount(sandbox_id).create_campaign(fields=fields,params=params))

In [ ]:
# setup a targeting
targeting={ 
    "geo_locations": {"countries":["US"]}, 
    "interests": [{id: 6003139266461, 'name': 'Movies'}]
}

In [ ]:
# init api connection
FacebookAdsApi.init(access_token=access_token)

# create an ad
fields = []
params = {
  'name': 'Sample Creative',
  'object_story_spec': {
      'page_id':page_id,
      'link_data':{
          'link':"https://www.facebook.com/1610651629161752/",
          'message':'try it out'
      },
  'targeting': targeting,
  'optimization_goal': 'REACH',
  'status': 'PAUSED',
  'budget': 1000,
  'bid_amount': 2,
  'billing_event': 'IMPRESSIONS',
  'start_time': '2021-04-01T06:00:00+0000'
  },
}
print("Got it done:\n", AdAccount(id).create_ad_creative(fields=fields,params=params))

The key for Heimbuchenthal is '561846'. I will use it for sharpening more target settings.

## extract insights

a reference on how to use facebook's APIs to extract insights https://vigneshwarilango.medium.com/connecting-facebook-business-manager-with-python-for-insights-6cd2fde20af2
